TODO:
- Add support for apartments  
- Handle or at least detect when Google has "detected unusual activty"  
- Search for ATT, Spectrum, and other Fiber providers.   

Google Geocoding free Tier:
- \\$200 worth of requests
- Geocoding: \\$5/1000 requests
- 40k requests for free/month

In [1]:
import googlemaps as gmap
import numpy
import requests
import concurrent.futures
import requests
import time
import os
import pickle

###########################################################################
# Config Fields
###########################################################################
# City-Wide Scan: 29.63144, -98.71617  29.30986, -98.26267
# Put coordinates for two opposing cordners of a rectangle to be scanned
coord1 = (29.30986, -98.26267)
coord2 = (29.63144, -98.71617)


#0.01 Low Res || 0.003 Med Res ||| 0.002 High Res
vscanRes = 0.003
hscanRes = 0.03

API_CONNECTIONS = 1
Scraper_Connections = 5
###########################################################################
# Config Fields
###########################################################################

latStart = min(coord1[0],coord2[0])
latEnd = max(coord1[0],coord2[0])
lonStart = min(coord1[1],coord2[1])
lonEnd = max(coord1[1],coord2[1])

In [2]:
API_Key = os.environ.get('GMaps_API_Key')
gmaps = gmap.Client(key=API_Key)

urlpre = "https://fiber.google.com/address?street_address="
urlmid = "&unit_number=&zip_code="
urlend = "&event_category=check%20address&event_action=submit&event_label=hero"
successurl = "https://fiber.google.com/signup/"
failstring = "Google Fiber isn’t available for this address"
failstring2 = "Enter your email address to get Google Fiber updates"
successstringHasService = "This address has a Google Fiber account"

In [3]:
def get_addr(coords):
    addr = gmaps.reverse_geocode(coords)
    return addr

def hasFiber(addr):
    address = ''
    zipcode = ''
    streetNum = ''
    route = ''
    for addComp in addr[0]['address_components']:
        if 'street_number' in addComp['types']:
            streetNum = addComp['long_name']
        if 'route' in addComp['types']:
            route = addComp['long_name']
        if 'postal_code' in addComp['types']:
            zipcode = addComp['long_name']
    address = streetNum + " " + route
    
    if not address.strip():
        return ""
    
    fullurl = urlpre + address.replace(" ","%20") + urlmid + zipcode + urlend
    r = requests.head(fullurl, allow_redirects=True)
    
    if successurl in r.url:
        print(str(address) + ","+ str(zipcode))
        print(r.url,"\n\n")
        return "AVAILABLE," + str(address) + ","+ str(zipcode)+"\n"
    else:
        return ""

def writeAddresses(filename, addresslist):
    with open(filename,'wb') as addressoutfile:
        pickle.dump(addresslist, addressoutfile)

def readAddresses(filename):
    with open(filename,'rb') as addressinfile:
        return pickle.load(addressinfile)

def setupIncOutFile():
    fileOutAvail = ""
    fileNum=1
    while not fileOutAvail:
        try:
            fileOutAvail  = open("dataAvail"+str(fileNum)+".csv", "x")
        except Exception as e:
            fileNum += 1
    fileOutAvail.write("Availability,StreetAddress,Zipcode\n")
    return fileOutAvail

def setupMainOutFile():
    try:
        fileOutAvailMain  = open("dataAvail.csv", "a")
    except Exception as e:
        print("dataAvail.csv is not available for writing. \nTERMINATING")
        exit()
    return fileOutAvailMain

def reverseGeoCode(coordsList):
    addresses = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=API_CONNECTIONS) as executor:
        future_to_url = (executor.submit(get_addr, coords) for coords in coordsList)
        for future in concurrent.futures.as_completed(future_to_url):
            try:
                data = future.result()
            except Exception as exc:
                print(exc)
                data = False
            finally:
                if data:
                    addresses.append(data)
                print(str(len(addresses)),end="\r")
    return addresses

def testAddresses(addresses, filewriter1, filewriter2):
    tests = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=Scraper_Connections) as executor:
        future_to_url = (executor.submit(hasFiber, addr) for addr in addresses)
        for future in concurrent.futures.as_completed(future_to_url):
            try:
                data = future.result()
            except Exception as exc:
                print(exc)
                data = False
            finally:
                tests += 1
                if data:
                    filewriter1.write(data)
                    filewriter2.write(data)

                print(tests,end="\r")


def makeCoordsList():
    coordsList = []
    for lat in numpy.arange(latStart, latEnd, vscanRes): 
        for lon in numpy.arange(lonStart, lonEnd, hscanRes):
            coordsList.append((lat,lon))
    
    cont = input(f"Starting {len(coordsList)} checks. \nContinue? [Y/N]: ")
    if 'y' not in cont.lower():
        raise UserWarning('Exit Early')
    return coordsList

def testFromListFile(filename):
    addresses = readAddresses(filename)
    fileOutAvail = setupIncOutFile()
    fileOutAvailMain = setupMainOutFile()

    testAddresses(addresses,fileOutAvail,fileOutAvailMain)

    fileOutAvail.close()
    fileOutAvailMain.close()

def revGeoCodeToFile(filename):
    coordslist = makeCoordsList()
    addresslist = reverseGeoCode(coordslist)
    writeAddresses(filename, addresslist)
    

In [4]:
#filename = input("Name for Scan Address List [*.pylist]:")
#revGeoCodeToFile(filename)
#testFromListFile(filename)

In [ ]:
filename = "WholeCityV003H03.pylist"
testFromListFile(filename)

359